# Projeto 1 - Ciência dos Dados

Nome: Felipe Schiavinato

Nome: Jerônimo Afrange

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/jeronimo/Desktop/Insper/2/CDADOS/p1-cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'bitcoin.xlsx'

In [4]:
dados_treinamento = pd.read_excel(filename)
dados_treinamento.head(5)

,Treinamento,Relevancia
0,do you think it is ethical of exchanges to ear...,3.0
1,@junkosu22993224: citi bank is the first major...,3.0
2,@wsbmod: it's time to decentralize wallstreetb...,2.0
3,@iamlluciana charlie the bitcoin fund manager ...,1.0
4,#btc #bitcoin #forex\nus stocks set for swoon ...,0.0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,@phorecrypto: a snapshot of #phore blockchain ...,2.0
1,@riena_smile: bottlepay launches twitter bitco...,3.0
2,@btc_archive: #bitcoin on target for 75k in t...,3.0
3,@grady_booch at this stage it seems vanishingl...,3.0
4,@ragnarly i have 3 kids. simply cant ask aroun...,2.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O nosso produto é o Bitcoin, e temos como objetivo analisar a opinião do público a respeito dessa criptomoeda específica. Classificamos a relevância dos Tweets com uma nota de 0 a 4, sendo 0 completamente irrelevante e 4 completamente relevante.

Consideramos como completamente irrelevantes Tweets que mencionam o nome Bitcoin mas não estão relacionados à criptomoeda, assim como Tweets de zombaria.

Consideramos como completamente relevantes Tweets que mencionam a criptomoeda, sejam, de fato, a respeito do nosso produto e autênticos.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [57]:

# extração das palavras
palavras_por_relevancia = [[],[],[],[],[]]

# elementos bloqueados
blacklist = "[!-—.:?;,#…']•()1234567890%$/\’+*" + '"“”'

# palavras inúteis
with open('useless_words.txt') as file:
    palavras_inuteis = file.read().split()

tweets = dados_treinamento['Treinamento']
relevancias = dados_treinamento['Relevancia']

# loop que passa tweet por tweet
for indice in range(len(tweets)):
    
    # cria constantes 'tweet_original' e 'relevancia'
    try: relevancia = int(relevancias[indice])
    except: continue
    tweet_original = tweets[indice]
    
    # remove links e nomes de usuário
    tweet_lista = tweet_original.split()
    tweet_sem_nomes_links = list()
    
    for palavra in tweet_lista:
        
        # remove letras maiúsculas
        palavra = palavra.lower()
                
        not_link = 'http' not in palavra
        not_usuario = '@' not in palavra
        
        if not_link and not_usuario:
            tweet_sem_nomes_links.append(palavra)
        
    tweet = ' '.join(tweet_sem_nomes_links)
    
    '''
    *** a partir desse ponto, 'tweet' não tem mais nomes de usuário nem links
    '''
    
    # remove caracteres bloqueados
    for carac in blacklist:
        
        if carac in tweet:
            tweet = tweet.split(carac)
            tweet = ' '.join(tweet)
                
    '''
    *** a partir desse ponto, 'tweet' não tem mais caracteres bloqueados
    '''
        
    # remove palavras inúteis
    tweet = tweet.split()
    
    for inutil in palavras_inuteis:
        while inutil in tweet:
            tweet.remove(inutil)
    
    tweet = ' '.join(tweet)
            
    '''
    *** a partir desse ponto, 'tweet' não tem mais palavras inúteis
    '''
                
    # remove espaços em branco duplos
    tweet = tweet.split()
    while '' in tweet: tweet.remove('')
    tweet = ' '.join(tweet)
    
    '''
    *** a partir desse ponto, 'tweet' não tem espaços em branco repetidos
    '''
    
    # adicionando as palavras à lista de palavras da respectiva relevância
    palavras_por_relevancia[relevancia] += tweet.split()
    

In [58]:

series_por_relevancia = list()

for palavras in palavras_por_relevancia:
    series_por_relevancia.append(pd.Series(palavras).value_counts(True))


In [60]:
todas_palavras = set()
for palavras in palavras_por_relevancia:
    for palavra in palavras:
        todas_palavras.add(palavra)

for palavra in todas_palavras:
    print(palavra)

stimmy
hourly
around
internet
weak
usdc
richer
legitimate
xauusd
etfs
lunch
posts
commerce
bubble
trend
embracing
hello
contest
behnam
hot
disguised
secretly
⬢
tron
fireblocks
criticism
asian
pinned
moving
accepted
kucoincryptogem
proposals
cannot
💪
success⚖️
lawsuit
tooth
goldma
presidents
flows
fuck
➖
didn
seems
wet
market
definition
efficient
tumblr
turn
party
claim
promised
ears
companies
guys
unique
nh
prnewswire
environment
info
first
sort
mined
receiving
banking
large
sotheby
robby
stellar
milli
formally
oklg
usatoday
revolution
volatile
pushing
cheaper
markets
graf
wi
state
overhaul
psychology
divergence
billionaire
instead
usually
popular
dol
mainstream
♥️
af
nigerians
together
detail
shopping
d
worlwide
narratives
mars
neighbour
view
pepe
partners
🔗
currency
crypto
builds
second
lead
smacontract
historic
weighted
week
tried
load
bonuses
selling
system
masses
reputable
collectible
eeenf
🌎
massive
footprint
mcallister
technologies
gic
hodlers
cryptoa
pop
stonk
grant
volume
nons

google
concentrated
machines
gold
ms
pouring
indicted
genuine
keep
suisse
dead
🚀🚀🚀🚀
hprice
en
allows
beta
death
manage
⚡
fdf
seem
problems
🔥
🤩
jambos
power
wt
gamblingtwitter
hawks
xvg
boils
concern
sisters
pattern
phone
exclusive
brought
due
homeowner
keiser
dm
po&amp
worse
silva
girl
slayer
irony
👇👇🔥
man
asset
affiliatemarketing
remarkably
🖖
fight
b
meant
organic
th
business
xbt
via
social
balances
sw
continues
wti
widows
drake
efi
six
filing
อีสานเทรดเดอร์
holy
🐐🛡️
retirement
videos
hiding
enjin
nft
shilling
pope
stimulous
fees
sent
enables
candles
especially
wealthiest
withdraw
&lt
quantity
enjoyable
addresses
story
record
berent
weird
earth
locked
stmx
mach
december
😁
pak
meitu
peer
xrp
quote
vaneck
swoon
parked
§ats
cancel
wiping
binancesmartchain
freedom
status
forextrader
gedt
😢
shoulder
expensive
➕
checks
active
🧡
davidson
roughly
twice
bullish
visa
question
deeplearning
anonymity
happens
bull
looking
shill
times
f
weakest
integrate
un
atari
diversification
quip
focused
rejoic

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**